# Self-Drive Cars' Sentiment Analysis
13.01.2020



In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hassansherwani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import spacy 

nlp = spacy.load("en_core_web_sm") # preload model

In [4]:
# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 2)-Loading data

In [5]:
#filename='tweets.csv'

In [6]:
#df = pd.read_csv(filename, delimiter='\t', header=None,names=['TWEET ID','TIMESTAMP','UTC DATE','USER ID','SCREEN NAME','NAME','Q','TEXT'])

In [7]:
#df_label=df[['TEXT','UTC DATE']]

### save file as pickle

In [8]:
#import pickle
#df_label.to_pickle('entity_data.pkl')
#df_label.to_csv('entity_data.csv',index=False)

### Loading saved csv file

In [9]:
filename='entity_data.csv'

In [10]:
l=pd.read_csv('entity_data.csv')
l.shape

(455728, 2)

In [11]:
l.head()

,TEXT,UTC DATE
0,TEXT,UTC DATE
1,"Another reason I failed in love is I'm poor,I ...",2018-07-31 23:13:47
2,2019 Mercedes-Benz C 300 and AMG C 43 First Dr...,2018-07-31 23:30:00
3,"Happy birthday to the Audi chasing maniac, hop...",2018-07-31 22:52:45
4,Come see Mercedes Benz stadium in Atlanta...,2018-07-31 23:02:25


**convert this datatime to month or year **

# 3)- Getting organization names

### 3.1)- Start with Google

#### Experiment

In [12]:
list_company = ['Google']

In [13]:
for company in list_company:
    l[company] = l.astype(str).sum(axis=1).str.contains(company)

In [14]:
l.head()

,TEXT,UTC DATE,Google
0,TEXT,UTC DATE,False
1,"Another reason I failed in love is I'm poor,I ...",2018-07-31 23:13:47,False
2,2019 Mercedes-Benz C 300 and AMG C 43 First Dr...,2018-07-31 23:30:00,False
3,"Happy birthday to the Audi chasing maniac, hop...",2018-07-31 22:52:45,False
4,Come see Mercedes Benz stadium in Atlanta...,2018-07-31 23:02:25,False


In [15]:
l.Google.value_counts()

False    448641
True       7087
Name: Google, dtype: int64

In [16]:
len(l.loc[l.TEXT.str.contains('Google',na=False)])

7087

In [17]:
l.loc[l.TEXT.str.contains('Google',na=False)]

,TEXT,UTC DATE,Google
433,Self-driving taxis will take half of US drivin...,2018-10-14 00:34:15,True
463,Companies outside of #tech have a key advantag...,2018-10-28 14:07:20,True
913,"Must hold CNN, Apple, Google's feet 2 the fire...",2016-11-02 16:20:30,True
1060,Soooo maybe Google is building its own self-dr...,2016-02-13 03:05:22,True
1066,"Exclusive: In boost to self-driving cars, U.S....",2016-02-13 10:40:27,True
...,...,...,...
453475,"Apple joins Tesla, Google, BMW, Ford, Honda an...",2017-04-19 00:01:01,True
454174,"IBM is your dad, and Google is his younger bro...",2016-11-29 23:02:07,True
454474,#SEO #entrepreneur Sell #1PH #electric #motors...,2016-03-15 17:06:06,True
454499,"Interesting to me that Google, Lyft, General M...",2016-03-15 20:02:13,True


#### Experiment round 2

To add 1 and 0

In [18]:
l['google'] = l['TEXT'].str.contains('Google', case=False).astype(int)

In [19]:
l.head()

,TEXT,UTC DATE,Google,google
0,TEXT,UTC DATE,False,0
1,"Another reason I failed in love is I'm poor,I ...",2018-07-31 23:13:47,False,0
2,2019 Mercedes-Benz C 300 and AMG C 43 First Dr...,2018-07-31 23:30:00,False,0
3,"Happy birthday to the Audi chasing maniac, hop...",2018-07-31 22:52:45,False,0
4,Come see Mercedes Benz stadium in Atlanta...,2018-07-31 23:02:25,False,0


In [21]:
l.google.value_counts()

0    446996
1      8732
Name: google, dtype: int64

### 3.2)- Tesla

In [22]:
l['tesla'] = l['TEXT'].str.contains('Tesla', case=False).astype(int)

In [23]:
l.tesla.value_counts()

0    433127
1     22601
Name: tesla, dtype: int64

### 3.3)-General Motors

In [24]:
l['gm'] = l['TEXT'].str.contains('General Motors', case=False).astype(int)

In [25]:
l.gm.value_counts()

0    424017
1     31711
Name: gm, dtype: int64

### 3.4)-Apple

In [26]:
l['apple'] = l['TEXT'].str.contains('Apple', case=False).astype(int)

In [27]:
l.apple.value_counts()

0    451229
1      4499
Name: apple, dtype: int64

### 3.5)-Ford

In [28]:
l['ford'] = l['TEXT'].str.contains('Ford', case=False).astype(int)

In [29]:
l.ford.value_counts()

0    437204
1     18524
Name: ford, dtype: int64

### 3.6)-Daimler-Benz

In [30]:
l['daimler-benz'] = l['TEXT'].str.contains('Daimler-Benz', case=False).astype(int)

In [31]:
l['daimler-benz'].value_counts()

0    455677
1        51
Name: daimler-benz, dtype: int64

### 3.7)-Volkswagen

In [32]:
l['volkswagen'] = l['TEXT'].str.contains('Volkswagen', case=False).astype(int)

In [33]:
l.volkswagen.value_counts()

0    437404
1     18324
Name: volkswagen, dtype: int64

### 3.8)-BMW

In [34]:
l['bmw'] = l['TEXT'].str.contains('BMW', case=False).astype(int)

In [35]:
l.ford.value_counts()

0    437204
1     18524
Name: ford, dtype: int64

### 3.9)-Audi

In [ ]:
l['google'] = l['TEXT'].str.contains('Google', case=False).astype(int)

In [ ]:
l.ford.value_counts()

### 3.10)-Porsche

In [ ]:
l['google'] = l['TEXT'].str.contains('Google', case=False).astype(int)

In [ ]:
l.ford.value_counts()

### 3.11)-Opel

In [ ]:
l['google'] = l['TEXT'].str.contains('Google', case=False).astype(int)

In [ ]:
l.ford.value_counts()